In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [49]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
sys.path.append('../utils')
import Landing3DModel_UEN
import Landing3DCost
import Landing3DConstraints
from scipy.integrate import solve_ivp
from Scvx import Scvx

In [50]:
def euler_to_quaternion(roll, pitch, yaw):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return [qw, qx, qy, qz]

In [51]:
ix = 14
iu = 3
ih = 7
tf = 5
N = 50
delT = tf/N
max_iter = 30

In [52]:
qi = euler_to_quaternion(np.deg2rad(0),np.deg2rad(0),0)
xi = np.array([2, 1,0,6, 0.0,0,-1,    qi[0],qi[1],qi[2],qi[3],   0,0,0])
xf = np.array([0.75, 0,0,0, 0,0,-0.1,       1,0,0,0,   0,0,0])

myModel = Landing3DModel_UEN.Landing3D('Hello',ix,iu,delT,linearization="analytic")
myCost = Landing3DCost.Landing3D('Hello',ix,iu,N)
myConst = Landing3DConstraints.Landing3D('Hello',ix,iu)

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
u0[:,2] = x0[:,0] * 1

# model forward check

In [59]:
from Scvx_from_github import Rocket_Model_6DoF, Integrator
m = Rocket_Model_6DoF()
print(m.f_func(x0[0],u0[0]))
print(myModel.forward(x0[0],u0[0],discrete=False))

[[-0.02]
 [ 0.  ]
 [ 0.  ]
 [-1.  ]
 [-1.  ]
 [ 0.  ]
 [ 1.  ]
 [-0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 2.  ]
 [-0.  ]]
[[-0.02  0.    0.   -1.   -1.    0.    1.   -0.    0.    0.    0.    0.
   2.   -0.  ]]


# Linearization

In [73]:
print(m.B_func(x0[10],u0[10]))

[[-0.         -0.         -0.01      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.57142857  0.          0.        ]
 [ 0.          0.57142857  0.        ]
 [ 0.          0.          0.57142857]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.         -1.          0.        ]]


In [74]:
A,B = myModel.diff(x0[10],u0[10],discrete=False)
print(B)

[[-0.         -0.         -0.01      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.57142857  0.          0.        ]
 [ 0.          0.57142857  0.        ]
 [ 0.          0.          0.57142857]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.         -1.          0.        ]]


# discretization check

In [88]:
import time
idx = 0
start = time.time()
A,Bm,Bp,s,z = myModel.diff_discrete_foh(x0[0:N,:],u0)
end = time.time()
print(end - start)

0.036273956298828125


In [89]:
K = N+1
# state and input list
X = np.empty(shape=[m.n_x, K])
U = np.empty(shape=[m.n_u, K])

for i in range(K) :
    X[:,i] = x0[i]
    U[:,i] = u0[i]
    
integrator = Integrator(m, K)
start = time.time()
A_bar, B_bar, C_bar, S_bar, z_bar = integrator.calculate_discretization(
            X, U, 1)
end = time.time()
print(end - start)

0.3085758686065674


# S

In [79]:
print(np.reshape(S_bar[:,idx_check],(ix)))
print()
print(s[idx_check])

[-1.98750000e-03 -9.96697490e-03  0.00000000e+00 -9.00368089e-02
 -9.90115024e-02  0.00000000e+00  9.94689337e-02 -4.95839553e-05
  0.00000000e+00  9.95820970e-03  0.00000000e+00  0.00000000e+00
  1.98750000e-01  0.00000000e+00]

[-1.98750000e-03 -9.96697490e-03  0.00000000e+00 -9.00368089e-02
 -9.90115013e-02  0.00000000e+00  9.94689337e-02 -4.95840059e-05
  0.00000000e+00  9.95820987e-03  0.00000000e+00  0.00000000e+00
  1.98750000e-01  0.00000000e+00]


# z

In [82]:
print(np.reshape(z_bar[:,idx_check],(ix)))
print()
print(z[idx_check])

[ 1.98750000e-03  4.95046482e-03  0.00000000e+00  9.50176765e-02
 -1.64733474e-03  0.00000000e+00 -4.15738490e-05  7.43759549e-05
  0.00000000e+00 -1.49373148e-02  0.00000000e+00  0.00000000e+00
 -1.98750000e-01  0.00000000e+00]

[ 1.98750000e-03  4.95046481e-03  0.00000000e+00  9.50176772e-02
 -1.64733482e-03  0.00000000e+00 -4.15734201e-05  7.43760041e-05
  0.00000000e+00 -1.49373148e-02  0.00000000e+00  0.00000000e+00
 -1.98750000e-01  0.00000000e+00]


# B

In [85]:
idx_check = 0
# np.reshape(A_bar[:,idx_check],(ix,ix)) - A[idx_check]
print(np.reshape(B_bar[:,idx_check],(ix,iu),order='F'))
print()
print(Bm[idx_check])

[[ 0.00000000e+00  0.00000000e+00 -4.99999986e-04]
 [ 1.66707510e-03  0.00000000e+00  4.99206069e-06]
 [ 0.00000000e+00  1.66707095e-03  0.00000000e+00]
 [-1.67574949e-06  0.00000000e+00  1.66768442e-03]
 [ 2.50082297e-02  0.00000000e+00  1.65600296e-04]
 [ 0.00000000e+00  2.50079267e-02  0.00000000e+00]
 [-4.15860792e-05  0.00000000e+00  2.50240742e-02]
 [ 0.00000000e+00  0.00000000e+00 -8.29853342e-06]
 [ 0.00000000e+00  8.18935399e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.66664114e-03]
 [ 0.00000000e+00 -1.66665469e-03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  4.99999986e-02]
 [ 0.00000000e+00 -4.99999986e-02  0.00000000e+00]]

[[ 0.00000000e+00  0.00000000e+00 -5.00000000e-04]
 [ 1.66708006e-03  0.00000000e+00  4.97658380e-06]
 [ 0.00000000e+00  1.66707524e-03  0.00000000e+00]
 [-1.66401615e-06  0.00000000e+00  1.66768864e-03]
 [ 2.50082283e-02  0.00000000e+00  1.65603488e-04]
 [ 0.00000000e+00  2.50079270